In [1]:
import pandas as pd
import pymc3 as pm
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

The code below loads the Excel file and selects the data within the first sheet. 

In [154]:
x1 = pd.ExcelFile('../data/response_data.xlsx')  
x1.sheet_names
df = x1.parse("Sheet1", header = 17, usecols = [0,1,2,3,4,5,6,7])
df.head()

,Date,Contact,Length,Ethnicity,Response,Age,Meeting,Single
0,2015-05-01,NaN,15,W,1.0,38.0,1.0,NaN
1,2015-05-07,NaN,10,U,NaN,NaN,NaN,NaN
2,2015-05-15,NaN,11,W,NaN,30.0,NaN,1.0
3,2015-05-18,NaN,20,A,NaN,25.0,NaN,1.0
4,2015-05-18,NaN,9,U,NaN,NaN,NaN,NaN


In [155]:
#Change W to White, U to Unkown, L to Latina, A to Asian, AA to African American, SA to South Asian
df.Ethnicity.replace( 'U', 'Unknown', inplace = True)
df.Ethnicity.fillna( 'Unknown' , inplace = True)
df.Ethnicity.replace( 'W', 'White', inplace = True)
df.Ethnicity.replace( 'L', 'Latina', inplace = True)
df.Ethnicity.replace( 'A', 'Asian', inplace = True)

Define a function for returning percentages

In [156]:
def perc(x, y):
    z = x / y 
    return "{:.1%}".format(z)

## Response rate for all contacted 

In [157]:
perc(df.Response.sum(),  len(df))

'20.9%'

## Percentage of in person meetings 

In [158]:
perc(df.Meeting.sum(),  len(df))

'2.2%'

## Average age differential of people contacted 

In [159]:
avg_age = "%.0f" % df.Age.mean() # determines age rounds to int
age_dif = 37 - int(avg_age)  #subtracts age from avg
age_dif

4

## Response % from Whites

In [160]:
perc( df.Ethnicity.value_counts()['White'], len(df))

'19.8%'

## Response % from Latinas

In [161]:
perc( df.Ethnicity.value_counts()['Latina'], len(df))

'8.8%'

## Response % from Asians

In [162]:
perc( df.Ethnicity.value_counts()['Asian'], len(df))

'6.6%'

## Response % from Unknown

In [163]:
perc( df.Ethnicity.value_counts()['Unknown'], len(df))

'64.8%'

## Percentage of messages where they reached out first 

In [164]:
perc( df.Contact.sum(), len(df) )

'3.3%'

## Percentage of Whites initiating first contact

In [139]:
perc( df.loc[(df.Contact == 1) & (df.Ethnicity == 'White') , 'Contact'].sum(), df.Contact.sum() )

'66.7%'

## Percentage of Latinas initiating first contact

In [166]:
perc( df.loc[(df.Contact == 1) & (df.Ethnicity == 'Latina') , 'Contact'].sum(), df.Contact.sum() )

'0.0%'

## Percentage of Asians initiating first contact

In [167]:
perc( df.loc[(df.Contact == 1) & (df.Ethnicity == 'Asian') , 'Contact'].sum(), df.Contact.sum() )

'0.0%'

## Percentage of Unknown initiating first contact

In [165]:
perc( df.loc[(df.Contact == 1) & (df.Ethnicity == 'Unknown') , 'Contact'].sum(), df.Contact.sum() )

'33.3%'

## My response rate upon being messaged first 

In [124]:
perc( df.loc[df.Contact == 1, 'Response'].sum(), df.Contact.sum() )

'100.0%'

## Percentage of contacted accounts still active 

In [126]:
perc( df.Single.sum(), len(df) )

'28.6%'

## Percentage of contacted accounts still active and single 

In [137]:
perc( df.Single.sum() ,  len(df.Single.loc[ df.Single == 1 | 0] ))

'100.0%'